# DEMO 

Example code to compute FFDI from a zarr store of one of RCM simulations

To fix

The code reads the intake catalogues of the SSP370 runs to get a list of models to process.  This is not necessary but it is a convenient way to get a list of all the files to process

Added the code process all the files in the search 

only works with module load conda/analysis3-23.10 because of problem with units in DF call (out needs?)


#### required packages

In [ ]:
import intake
import xarray as xr
from matplotlib import pyplot as plt

import glob
import pathlib
import traceback
from datetime import datetime

from xclim.indices import (
    keetch_byram_drought_index,
    griffiths_drought_factor,
    mcarthur_forest_fire_danger_index
)


#### start a local Dask client

In [ ]:
from dask.distributed import Client
import dask

# Set configuration options
dask.config.set({
    'distributed.comm.timeouts.connect': '90s',  # Timeout for connecting to a worker
    'distributed.comm.timeouts.tcp': '90s',  # Timeout for TCP communications
})

from dask.distributed import Client
client = Client()
client

#### ignore warnings

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Obtain the desired catalogues of the simulations to processe

In [ ]:
catalogue_path='/g/data/ia39/catalogues'
mRuns = sorted(glob.glob(catalogue_path+'/*ssp370*.json'))
mRuns = sorted(glob.glob(catalogue_path+'/*BARRA*.json'))
print(len(mRuns))

In [ ]:
# test loading all the catalogues of the data
for file in mRuns: 
    print(file)
    cat1=intake.open_esm_datastore(file)

# Process one ensemble 

In [ ]:
# From one catalogue list save variables
mindex=0  # run to process now set a start
mtmp=mRuns[mindex].split('/')
model_name=mtmp[5].split('.')
print(model_name)
tmp=model_name[0].split('_'); ACS_model=tmp[1]

# now match extract RCM_GCM info 
match=tmp[1]+'_'+tmp[2]
print(match)

In [ ]:
%%time
# read in the historical catalogue to compute 
# will modified to use the same Zarr collection
if match != "BOM_ERA5": 
    catalogue_path='/g/data/ia39/catalogues'
    mHis = sorted(glob.glob(catalogue_path+'/*'+match+'*hist*.json'))
    print(len(mHis))
    print(mHis[0])
    cat1=intake.open_esm_datastore(mHis[0])
    pr_his = cat1.search(variable={'pr'}).to_dask(cdf_kwargs={'chunks': {'time': 365, 'lat': 691, 'lon': 886}})
    pr_his
# This is usually computed over a specific period
    pr_annual = pr_his.pr.resample(time="Y").sum().\
        sel(time=slice('1985-01-01', '2015-01-01')).mean("time").\
        assign_attrs(
        units="mm a-1", long_name="Annual average rainfall")
    pr_annual.load()
    plt.subplot(2,2,1); pr_annual.plot()


In [ ]:
#AGCD-05i_CSIRO_EC-Earth3_ssp370_r1i1p1f1_CCAM-v2203-SN_v1-r1_day.zarr/
dir='/scratch/xv83/rxm599/'
file=dir+model_name[0]+'.zarr'
print(file)
ds0 = xr.open_zarr(file)
ds0

if match == 'BOM_ERA5':  # This is usually computed over a specific period
    pr_annual = ds0.pr.resample(time="Y").sum().\
        sel(time=slice('1985-01-01', '2015-01-01')).mean("time").\
        assign_attrs( units="mm a-1", long_name="Annual average rainfall")

pr_annual.load()
pr_annual.plot()


# Compute FFDI

In [ ]:
def calculate_ffdi(pra, tasmaxa, pr_annual, hursmina, sfcWindmaxa):
    KBDI = keetch_byram_drought_index(pra, tasmaxa, pr_annual)
    DF = griffiths_drought_factor(pra, KBDI)
    FFDI = mcarthur_forest_fire_danger_index(DF, tasmaxa, hursmina, sfcWindmaxa)
    return FFDI.to_dataset(name='FFDI')

In [ ]:
t1='1975-01-01'; t2='2099-12-31'
pra=ds0.pr.sel(time=slice(t1, t2))
tasmaxa=ds0.tasmax.sel(time=slice(t1,t2))
hursmina=ds0.hursmin.sel(time=slice(t1,t2))
sfcWindmaxa=ds0.sfcWindmax.sel(time=slice(t1,t2))

KBDI = keetch_byram_drought_index(pra, tasmaxa, pr_annual) 
DF = griffiths_drought_factor(pra, KBDI)
FFDI = mcarthur_forest_fire_danger_index(DF, tasmaxa, hursmina, sfcWindmaxa)
ds = FFDI.to_dataset(name='FFDI')

In [ ]:
%%time
ds.to_zarr('/g/data/ia39/ncra/fire/'+model_name[0]+'_FFDI.zar',mode='w')
#tar -czvf compressed_zarr_store.tar.gz /path/to/zarr_store

## End here for BARRA

## Check the output

In [ ]:
# First 20 points are nan due to the way DI is computed (see XCLIM)
ds1=xr.open_zarr('/g/data/ia39/ncra/fire/'+model_name[0]+'_FFDI.zarr')
ds1.FFDI.isel(time=slice(19,None)).max('time',skipna=False).plot()
#ds1.FFDI.isel(lon=200,lat=300).plot()

In [ ]:
data=ds1.FFDI
d3=(data > 100 ).groupby('time.year').sum('time').to_dataset(name='days100')

In [ ]:
plt.subplot(2,2,1); d3.days100.mean('year').plot(levels=10)
plt.subplot(2,2,2); d3.days100.std('year').plot(levels=10)
plt.subplot(2,2,3); d3.days100.max('year').plot(levels=10)

In [ ]:
client.shutdown()